# Hypersonic Dataset Tutorial: Loading, Filtering, Plotting, and Saving Data from the OSIRIS-REx Re-entry

This tutorial goes over the basics of how to read, filter, and plot data collected during the atmospheric re-entry over Nevada and Utah of NASA's OSIRIS-REx sample return capsule.

A file containing data from a single station is included with this tutorial. The full  dataset can be downloaded from __[Google Drive](https://drive.google.com/drive/folders/1QKHUzdT0et6xpKXm8LFOmHJ9_gx0J2ib?usp=sharing)__ (contact Sarah Popenhagen at spopen@hawaii.edu for folder access).

The OREX data collection campaign was a collaboration between multiple institutions that resulted in numerous publications, a few of which are listed below.

__[Silber, E.A. *et al.* Geophysical Observations of the 2023 September 24 OSIRIS-REx Sample Return Capsule Reentry. *Planet. Sci. J.* **5**, 213 (2024).](https://doi.org/10.3847/PSJ/ad5b5e)__

__[Silber, E.A. & Bowman, D.C. Along‐Trajectory Acoustic Signal Variations Observed During the Hypersonic Re‐Entry of the OSIRIS‐REx Sample Return Capsule. *Seismol. Res. Lett.* **XX**, 1-13 (2025).](https://doi.org/10.1785/0220250014)__

__[Fernando, B. *et al.* Seismoacoustic measurements of the OSIRIS-REx re-entry with an off-grid Raspberry PiShake. *Seismica* **3**, 1 (2024).](https://doi.org/10.26443/seismica.v3i1.1154)__

__[KC, R.J. *et al.* Acoustic Observations of the OSIRIS-REx Sample Return Capsule Re-Entry from Wendover Airport. *Seismol. Res. Lett.* **XX**, 1-14 (2025).](https://doi.org/10.1785/0220250019)__

## Section 0: Prerequisites and Imports
The following cell includes the imports necessary to run this example.

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime, timezone

import quantum_inferno.plot_templates.plot_base as ptb
from quantum_inferno.plot_templates.plot_templates import plot_mesh_wf_vert
from quantum_inferno.styx_stx import tfr_stx_fft
from quantum_inferno.utilities.rescaling import to_log2_with_epsilon

## Section 1: Loading the Dataset

In the following cell, we'll define the path to the dataset. By default, this path will point to the single-station subset of data included with this tutorial.

After you've completed the tutorial with this file, feel free to download the full dataset and use the last line of the cell to change the 'PATH_TO_NPZ' variable to point to its location on your device.

In [ ]:
TUTORIAL_NPZ_FILE_NAME = "OREX_tutorial.npz"
CURRENT_DIRECTORY = os.getcwd()
PATH_TO_TUTORIAL_NPZ = os.path.join(CURRENT_DIRECTORY, TUTORIAL_NPZ_FILE_NAME)
PATH_TO_NPZ = PATH_TO_TUTORIAL_NPZ
# PATH_TO_NPZ = "<insert path to 'orex_best_mics_800hz_1024pt.npz' on your device here>"

Once we have the location of the file, we can read the data by passing `PATH_TO_NPZ` and allow_pickle=True to the numpy function __[np.load()](https://numpy.org/doc/stable/reference/generated/numpy.load.html)__.

In [ ]:
orex_npz: np.ndarray = np.load(PATH_TO_NPZ, allow_pickle=True)
orex_npz

In the output of the previous cell, we see the NPZ file's keys: "station_ids", "station_labels", "station_wf", and "station_epoch_s". For ease of use, we'll define a class containing these keys, as well as a few others that will become useful.

In [ ]:
class OREXLabels:
    """
    A class containing the keys used in the OSIRIS-REx NPZ file.
    """
    def __init__(
            self,
            station_id: str = "station_ids",
            station_label: str = "station_labels",
            station_make: str = "station_make",
            station_model: str = "station_model_number",
            station_network: str = "deployment_network",
            audio_data: str = "station_wf",
            audio_epoch_s: str = "station_epoch_s",
            audio_fs: str = "audio_sample_rate_nominal_hz",
            event_id: str = "event_id",
    ):
        """
        Defaults should be left in place for most uses.
        :param station_id: key associated with the unique ID string of the station used to record the signal
        :param station_label: key associated with the descriptive label string of the station
        :param station_make: key associated with the recording smartphone's make
        :param station_model: key associated with the recording smartphone's model
        :param station_network: key associated with the network on which the smartphone was deployed
        :param audio_data: key associated with the audio waveform of the signal
        :param audio_epoch_s: key associated with the time array of the audio waveform in epoch seconds
        :param audio_fs: key associated with the sample rate of the audio data in Hertz
        :param event_id: key associated with the unique ID string of the event associated with the signal
        """
        self.station_id = station_id
        self.station_label = station_label
        self.station_make = station_make
        self.station_model = station_model
        self.station_network = station_network
        self.audio_data = audio_data
        self.audio_epoch_s = audio_epoch_s
        self.audio_fs = audio_fs
        self.event_id = event_id

ds_labels = OREXLabels()

Using these labels as column names, we'll construct a pandas DataFrame to organize the data and metadata from the NPZ file.

In [ ]:
orex_df = pd.DataFrame()
for field in orex_npz.files:
    field_element = orex_npz[field]
    if len(field_element.shape) > 1:
        field_element = [field_element[i, :] for i in range(field_element.shape[0])]
    orex_df[field] = field_element
orex_df.head(5)

We'll also include ground truth values for the `event_id` and `audio_fs` fields. All audio data in the OREX NPZ file were recorded at 800 Hz, and all are associated with the OSIRIS-REx reentry.

In [ ]:
audio_fs_hz: float = 800.0
orex_event_id: str = "OREX"
n_signals = len(orex_df)
orex_df[ds_labels.audio_fs] = [audio_fs_hz] * n_signals
orex_df[ds_labels.event_id] = [orex_event_id] * n_signals
orex_df.head(5)


The descriptive strings associated with the `station_label` field contain information about the general location of the station and the model of the phone. 

For example, the station label "WEND S22-01" indicates that the associated data were recorded by a Samsung Galaxy S22 smartphone as part of the Wendover deployment.

With the labels now easily accessible, we'll print out some metadata about the recording(s) in our dataset in the next cell.

Notice how the desired fields are accessed using the column names stored in `ds_labels`.

In [ ]:
n_signals = len(orex_df)
print(f"This dataset contains {n_signals} recording(s) from the OSIRIS-REx atmospheric reentry:")
for idx in orex_df.index:
    signal_length_s = orex_df[ds_labels.audio_epoch_s][idx][-1] - orex_df[ds_labels.audio_epoch_s][idx][0]
    station_label_info = orex_df[ds_labels.station_label][idx].split(" ")
    deployment_net = station_label_info[0]
    smartphone_model = station_label_info[-1].split("-")[0]
    print(f"\nStation {orex_df[ds_labels.station_id][idx]}:")
    print(f"\tSignal length: {signal_length_s} s")
    print(f"\tSmartphone model: {smartphone_model}")
    print(f"\tDeployment network: {deployment_net}")

To facillitate filtering of the dataset, we'll extract this information from the station labels and add it to our DataFrame using list comprehension and the functions defined in the next cell.

In [ ]:
def get_station_model(station_label_string):
    return station_label_string.split(" ")[-1].split("-")[0]

def get_station_network(station_label_string):
    return station_label_string.split(" ")[0]

orex_df[ds_labels.station_model] = [get_station_model(sls) for sls in orex_df[ds_labels.station_label]]
orex_df[ds_labels.station_network] = [get_station_network(sls) for sls in orex_df[ds_labels.station_label]]
orex_df.head(5)


## Section 2: Filtering the Dataset

The dataset can be filtered easily using any of the included fields. For example, you could select a subset containing only data from stations included in the "WEND" network deployed southwest of Wendover, UT during the event.

```python
wend_df = orex_df[orex_df[ds_labels.station_network] == "WEND"]]
```

For the tutorial, we've selected a subset of the dataset containing only the data recorded by a single station. To illustrate the methods, however, we'll filter the tutorial dataset to select only the data associated with our example station. 

If you're using the full dataset, this will create a subset of the data identical to the tutorial dataset.

In [ ]:
example_station_id: str = "1637622001"
example_df = orex_df[orex_df[ds_labels.station_id] == example_station_id]
example_df

## Section 3: Plotting OREX Data

In the next cell, we'll define a function to plot the audio data from a single station in the time domain. Read through the comments in the function for a detailed explanation of each step.

In [ ]:
# fontsize to use in plots
fontsize = 12

def demean_norm(signal: np.ndarray) -> np.ndarray:
    signal = signal - np.nanmean(signal)
    return signal / np.nanmax(np.abs(signal))

def single_station_time_domain_plot(station_df, ds_labels):
    station_idx = station_df.index[0]
    event_id = station_df[ds_labels.event_id][station_idx]
    station_id = station_df[ds_labels.station_id][station_idx]
    station_net = station_df[ds_labels.station_network][station_idx]
    station_label = station_df[ds_labels.station_label][station_idx]
    print(f"\nEvent name: {event_id}, deployment network: {station_net}, station ID: {station_id}")
    # To make the visualization cleaner, we'll plot the audio data against time relative to the first sample
    t0 = station_df[ds_labels.audio_epoch_s][station_idx][0]
    relative_time_s = station_df[ds_labels.audio_epoch_s][station_idx] - t0
    date_string = (datetime.fromtimestamp(t0, tz=timezone.utc)).strftime("%Y-%b-%d %H:%M:%S")
    # We'll also demean and normalized the audio waveform to the range [-1, 1]

    fig, ax = plt.subplots(1, 1, figsize=(7, 5))
    fig.suptitle(f"Audio waveform from {event_id}\nStation {station_id} ({station_label})", fontsize=fontsize + 2)
    ax.plot(relative_time_s, demean_norm(station_df[ds_labels.audio_data][station_idx]), lw=1, color="k")
    ax.set(xlim=(0.0, relative_time_s[-1]), ylim=(-1.1, 1.1))
    ax.tick_params(axis="y", labelsize="large", left=True, labelleft=True)
    ax.tick_params(axis="x", labelsize="large", bottom=True, labelbottom=True)
    ax.set_ylabel("Norm", fontsize=fontsize)
    ax.set_xlabel(f"Time (s) since {date_string} UTC", fontsize=fontsize)

In the tutorial subset, only the data from one station are included, but we'll loop through the indices anyway to illustrate how to generate plots for all stations in a DataFrame.

In [ ]:
example_df = example_df.sort_values(by=ds_labels.station_label)
event_stations = example_df[ds_labels.station_id]
for station_id in event_stations:
    example_station_df = example_df[example_df[ds_labels.station_id] == station_id]
    single_station_time_domain_plot(station_df=example_station_df, ds_labels=ds_labels)

We can also generate time-frequency representations of the data. In the next cell, we'll use our example audio signal to illustrate how to do this quickly and easily using functions from the quantum_inferno module.

In [ ]:
# Order sets the atom resolution
order_number_input: int = 3

# loop through the rows
for row_index in example_df.index:
    fs = example_df[ds_labels.audio_fs][row_index]
    # Averaging window sets lowest frequency of analysis (lower passband edge).
    fft_duration_ave_window_points_pow2 = 8*1024
    print(f'Averaging period = {fft_duration_ave_window_points_pow2 / fs} s')
    frequency_resolution_fft_hz = fs / fft_duration_ave_window_points_pow2
    # Normalize the audio waveform and time array
    sig_wf = demean_norm(example_df[ds_labels.audio_data][row_index])
    t0 = example_df[ds_labels.audio_epoch_s][row_index][0]
    sig_time_s = example_df[ds_labels.audio_epoch_s][row_index] - t0
    # Compute Stockwell transform
    [stx_complex, _, frequency_stx_hz, _, _] = tfr_stx_fft(
        sig_wf=sig_wf,
        time_sample_interval=1 / fs,
        frequency_min=frequency_resolution_fft_hz,
        frequency_max=fs / 2,
        scale_order_input=order_number_input,
        n_fft_in=fft_duration_ave_window_points_pow2,
        is_geometric=True,
        is_inferno=False,
    )

    stx_power = 2 * np.abs(stx_complex) ** 2
    mic_stx_bits = to_log2_with_epsilon(np.sqrt(stx_power))

    # Select plot frequencies
    fmin_plot = 4 * frequency_resolution_fft_hz  # Octaves above the lowest frequency of analysis
    fmax_plot = fs / 2  # Nyquist

    # Plot the STX
    wf_base = ptb.WaveformPlotBase(
        f"{example_df[ds_labels.station_id][row_index]} ({example_df[ds_labels.station_label][row_index]})", 
        f"Stockwell transform of {example_df[ds_labels.event_id][row_index]} audio data")
    wf_panel = ptb.WaveformPanel(sig_wf, sig_time_s)
    mesh_base = ptb.MeshBase(sig_time_s, frequency_stx_hz, frequency_hz_ymin=fmin_plot, frequency_hz_ymax=fmax_plot)
    mesh_panel = ptb.MeshPanel(mic_stx_bits, colormap_scaling="range", cbar_units="log$_2$(Power)")
    stx = plot_mesh_wf_vert(mesh_base, mesh_panel, wf_base, wf_panel)

    plt.show()

## Section 4: Saving OREX Data

We can save a subset of the data to a new PKL or NPZ file. This can be useful in cases where memory is limited and the full dataset is not required.

We can save any subset (data from all stations in a network, data recorded by S22 phones, etc.), but we'll stick with our example subset of data from only station 1637622001, and only save the fields that are included in the original NPZ dataset file.

The code in the next cell can be modified easily to save data from a different station by simply changing the value of `station_id_to_save`.

In [ ]:
station_id_to_save: str = "1637622001"
columns_to_save = [ds_labels.audio_data, ds_labels.audio_epoch_s, ds_labels.station_id, ds_labels.station_label]

# check if the station id is in the subset
station_ids = np.unique(orex_df[ds_labels.station_id])
if station_id_to_save in station_ids:
    # filtering
    subset_to_save = orex_df[orex_df[ds_labels.station_id] == station_id_to_save]
    subset_to_save = subset_to_save[columns_to_save]
    # set filenames
    output_filename = f"OREX_{station_id_to_save}"
    output_path_pkl = os.path.join(CURRENT_DIRECTORY, f"{output_filename}.pkl")
    output_path_npz = os.path.join(CURRENT_DIRECTORY, f"{output_filename}.npz")
    subset_to_save.to_pickle(output_path_pkl)
    print(f"OREX data from station {station_id_to_save} saved to: {output_path_pkl}")
    np.savez(output_path_npz, 
             **{ds_labels.audio_data: subset_to_save[ds_labels.audio_data], 
                ds_labels.audio_epoch_s: subset_to_save[ds_labels.audio_epoch_s], 
                ds_labels.station_id: subset_to_save[ds_labels.station_id], 
                ds_labels.station_label: subset_to_save[ds_labels.station_label]})
    print(f"OREX data from station {station_id_to_save} saved to: {output_path_npz}")
else:
    print(f"Station {station_id_to_save} not found in OREX data file. No data saved.")


This concludes the tutorial. For more details on data from OSIRIS-REx, see the references listed at the beginning of the tutorial.